In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely import LineString, Point
from shapely.geometry import box

In [ ]:

gdf = gpd.read_file("boylston.geojson")
if gdf.crs.is_geographic:
    gdf = gdf.to_crs(epsg=3857)

line = gdf.geometry.iloc[0]
total_length = line.length

cmx_df = pd.read_csv("cm_final.csv")

step_lengths = cmx_df["cmx"].values / 100.0
cmy = cmx_df["cmy"]
id = cmx_df["id"]

cmx_with_start = np.insert(step_lengths * 100, 0, 0)

cumulative_dists = np.insert(np.cumsum(step_lengths), 0, 0.0)

points = [line.interpolate(dist) for dist in cumulative_dists]
print(points)

points_gdf = gpd.GeoDataFrame(geometry=points, crs=gdf.crs)
print(points_gdf)
points_gdf["cmx"] = cmx_with_start
points_gdf["cmy"] = cmy
points_gdf["id"] = id

points_gdf.to_file("points-spaced-by-cmx.geojson", driver="GeoJSON")


In [ ]:
step_lengths_y = cmx_df["cmy"].values / 100.0

combined = np.column_stack((step_lengths, step_lengths_y))

bbox_geoms = []

for point, (dx, dy) in zip(points_gdf.geometry, combined):
    minx = point.x - dx / 2
    maxx = point.x + dx / 2
    miny = point.y - dy / 2
    maxy = point.y + dy / 2
    bbox = box(minx, miny, maxx, maxy)
    bbox_geoms.append(bbox)

bbox_gdf = gpd.GeoDataFrame(geometry=bbox_geoms, crs=gdf.crs)

bbox_gdf['id'] = id
print(bbox_gdf)
bbox_gdf.to_file("bboxes_NEW.geojson", driver="GeoJSON")
bbox_gdf.plot()